# 🎨 NeMo Data Designer: Product Information Dataset Generator with Q&A using Open Router's Distillation endpoints API

#### 📚 What you'll learn

This notebook demonstrates how to use NeMo Data Designer to create a synthetic dataset of product information with corresponding questions and answers using OpenRouter's distillation endpoints

### Install the package if not installed

In [1]:
# !pip install data-designer==0.3.4
# !pip install dotenv==0.9.9

In [ ]:
from dotenv import load_dotenv
import os
env = load_dotenv()

Open_Router_Api_Key = os.getenv('OPEN_ROUTER')
Base_URL = "https://openrouter.ai/api/v1"

### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [3]:
from data_designer.essentials import (
    CategorySamplerParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    LLMTextColumnConfig,
    SamplerColumnConfig,
    SamplerType,
    ModelProvider,
    UniformSamplerParams,
    Score,
    ModelConfig,
    ChatCompletionInferenceParams,
    ExpressionColumnConfig,
    LLMStructuredColumnConfig,
    BernoulliSamplerParams,
    LLMJudgeColumnConfig,
)

# If we want debugging - uncomment the following
# from data_designer.logging import configure_logging, LoggingConfig, OutputConfig
# import sys

# # Enable debug logging with structured JSON output to see full API call details
# configure_logging(
#     LoggingConfig(
#         logger_configs=[LoggingConfig.debug().logger_configs[0]],
#         output_configs=[OutputConfig(destination=sys.stderr, structured=True)],  # Use JSON format
#         root_level="DEBUG"
#     )
# )


/Users/asteiner/Documents/Git_Repos/data_designer_open_router/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Try to call the OpenRouter endpoint first

In [4]:
from openai import OpenAI

client = OpenAI(
  base_url=Base_URL,
  api_key=Open_Router_Api_Key,
)

# First API call with reasoning
completion = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[{"role": "user", "content": "what is 1+1"}],
    max_tokens=100,
    stream=False,
    extra_body={"reasoning": {"enabled": False}}
)

reasoning = getattr(completion.choices[0].message, "reasoning", None)
if reasoning:
  print(reasoning)
  print('---')
print(completion.choices[0].message.content)


The answer to **1 + 1** is **2**.  

This is a fundamental arithmetic operation in the decimal (base-10) number system. Here's why:  
- Adding the quantity of one item to another item results in **two** items total.  

*(Note: In some advanced contexts like binary logic or abstract algebra, "1 + 1" might yield a different result, but in standard arithmetic, it is universally **2**.)*


### ⚙️ Initialize the NeMo Data Designer Client

- `DataDesigner` is responsible for submitting generation requests to the microservice.


In [ ]:
model_provider = ModelProvider(
    name = "deepinfra",
    endpoint = "https://openrouter.ai/api/v1/",
    provider_type = "openai",
    api_key = Open_Router_Api_Key,
    extra_body={
        "provider": {
            "enforce_distillable_text": True,
            # optionally, prefer DeepInfra endpoints
            "only": ["deepinfra"]
        }
    }
)

data_designer_client = DataDesigner(model_providers=[model_provider])


### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see [the model config docs](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/configure-models.html) for more details).


In [6]:
model_alias="nemotron-3-nano-30b-a3b"

inference_parameters = ChatCompletionInferenceParams(
    temperature=0.5,
    top_p=0.9,
    max_tokens=10000,
    max_parallel_requests=10,  # Number of concurrent workers
    extra_body={
        "reasoning": {"enabled": False}
    },
)

model_configs = [
    ModelConfig(
        alias=model_alias,
        model="nvidia/nemotron-3-nano-30b-a3b",
        provider="deepinfra",
        inference_parameters=inference_parameters
        )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [7]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🎲 Adding Sampler Columns

- Sampler columns offer non-LLM based generation of synthetic data.

- They are particularly useful for **steering the diversity** of the generated data, as we demonstrate below.


In [8]:
import string
from pydantic import BaseModel
from pydantic import Field

# Define product category options
config_builder.add_column(
    SamplerColumnConfig(
        name="category",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home Appliances",
                "Groceries",
                "Toiletries",
                "Sports Equipment",
                "Toys",
                "Books",
                "Pet Supplies",
                "Tools & Home Improvement",
                "Beauty",
                "Health & Wellness",
                "Outdoor Gear",
                "Automotive",
                "Jewelry",
                "Watches",
                "Office Supplies",
                "Gifts",
                "Arts & Crafts",
                "Baby & Kids",
                "Music",
                "Video Games",
                "Movies",
                "Software",
                "Tech Devices",
            ]
        ),
    )
)

# Define price range to seed realistic product types
config_builder.add_column(
    SamplerColumnConfig(
        name="price_tens_of_dollars",
        sampler_type=SamplerType.UNIFORM,
        params=UniformSamplerParams(low=1, high=200),
    )
)

config_builder.add_column(
    ExpressionColumnConfig(
        name="product_price",
        expr="{{ (price_tens_of_dollars * 10) - 0.01 | round(2) }}",
        dtype="float",
    )
)

# Generate first letter for product name to ensure diversity
config_builder.add_column(
    SamplerColumnConfig(
        name="first_letter",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(values=list(string.ascii_uppercase)),
    )
)

# Determine if this example will include hallucination
config_builder.add_column(
    SamplerColumnConfig(
        name="is_hallucination",
        sampler_type=SamplerType.BERNOULLI,
        params=BernoulliSamplerParams(p=0.5),
    )
)

DataDesignerConfigBuilder(
    sampler_columns: [
        "category",
        "price_tens_of_dollars",
        "first_letter",
        "is_hallucination"
    ]
    expression_columns: ['product_price']
)

## 🦜 LLM-generated columns

- When prompting the LLM, we can use Jinja templating to reference other columns in the dataset.

- As we see below, nested json fields can be accessed using dot notation.


In [9]:
# Define product information structure
class ProductInfo(BaseModel):
    product_name: str = Field(
        ..., description="A realistic product name for the market."
    )
    key_features: list[str] = Field(
        ..., min_length=1, max_length=3, description="Key product features."
    )
    description: str = Field(
        ...,
        description="A short, engaging description of what the product does, highlighting a unique but believable feature.",
    )
    price_usd: float = Field(..., description="The stated price in USD.")


# Generate product information
config_builder.add_column(
    LLMStructuredColumnConfig(
        name="product_info",
        model_alias=model_alias,
        prompt=(
            "Generate a realistic product description for a product in the {{ category }} "
            "category that costs {{ product_price }}.\n"
            "The name of the product MUST start with the letter {{ first_letter }}.\n"
        ),
        output_format=ProductInfo,
    )
)

# Generate user questions about the product
config_builder.add_column(
    LLMTextColumnConfig(
        name="question",
        model_alias=model_alias,
        prompt=("Ask a question about the following product:\n\n {{ product_info }}"),
    )
)


# Generate answers to the questions
config_builder.add_column(
    LLMTextColumnConfig(
        name="answer",
        model_alias=model_alias,
        prompt=(
            "{%- if is_hallucination == 0 -%}\n"
            "<product_info>\n"
            "{{ product_info }}\n"
            "</product_info>\n"
            "{%- endif -%}\n"
            "User Question: {{ question }}\n"
            "Directly and succinctly answer the user's question.\n"
            "{%- if is_hallucination == 1 -%}\n"
            "Make up whatever information you need to in order to answer the user's request.\n"
            "{%- endif -%}"
        ),
    )
)

DataDesignerConfigBuilder(
    sampler_columns: [
        "category",
        "price_tens_of_dollars",
        "first_letter",
        "is_hallucination"
    ]
    llm_text_columns: ['question', 'answer']
    llm_structured_columns: ['product_info']
    expression_columns: ['product_price']
)

## 🔍 Quality Assessment: LLM-as-a-Judge

When generating our synthetic dataset, we need to determine the quality of the generated data \
We use the LLM-as-a-Judge strategy to do this.

To do so, we need to define the rubric that the LLM should use to assess generation quality along with a prompt
that provides relavant instructions.


In [10]:
# Define evaluation rubrics for answer quality
CompletenessRubric = Score(
    name="Completeness",
    description="Evaluation of AI assistant's thoroughness in addressing all aspects of the user's query.",
    options={
        "Complete": "The response thoroughly covers all key points requested in the question, providing sufficient detail to satisfy the user's information needs.",
        "PartiallyComplete": "The response addresses the core question but omits certain important details or fails to elaborate on relevant aspects that were requested.",
        "Incomplete": "The response significantly lacks necessary information, missing major components of what was asked and leaving the query largely unanswered.",
    },
)

AccuracyRubric = Score(
    name="Accuracy",
    description="Evaluation of how factually correct the AI assistant's response is relative to the product information.",
    options={
        "Accurate": "The information provided aligns perfectly with the product specifications without introducing any misleading or incorrect details.",
        "PartiallyAccurate": "While some information is correctly stated, the response contains minor factual errors or potentially misleading statements about the product.",
        "Inaccurate": "The response presents significantly wrong information about the product, with claims that contradict the actual product details.",
    },
)


# Evaluate answer quality
config_builder.add_column(
    LLMJudgeColumnConfig(
        name="llm_answer_metrics",
        model_alias=model_alias,
        prompt=(
            "<product_info>\n"
            "{{ product_info }}\n"
            "</product_info>\n"
            "User Question: {{question }}\n"
            "AI Assistant Answer: {{ answer }}\n"
            "Judge the AI assistant's response to the user's question about the product described in <product_info>."
        ),
        scores=[CompletenessRubric, AccuracyRubric],
    )
)


# Extract metric scores for easier analysis
config_builder.add_column(
    ExpressionColumnConfig(
        name="completeness_result",
        expr="{{ llm_answer_metrics.Completeness.score }}",
    )
)

config_builder.add_column(
    ExpressionColumnConfig(
        name="accuracy_result",
        expr="{{ llm_answer_metrics.Accuracy.score }}",
    )
)

DataDesignerConfigBuilder(
    sampler_columns: [
        "category",
        "price_tens_of_dollars",
        "first_letter",
        "is_hallucination"
    ]
    llm_text_columns: ['question', 'answer']
    llm_structured_columns: ['product_info']
    llm_judge_columns: ['llm_answer_metrics']
    expression_columns: ['product_price', 'completeness_result', 'accuracy_result']
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [11]:
# Preview a few records
preview = data_designer_client.preview(config_builder)

[10:00:05] [INFO] 📸 Preview generation in progress
[10:00:05] [INFO] ✅ Validation passed
[10:00:05] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[10:00:05] [INFO] 🩺 Running health checks for models...
[10:00:05] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'deepinfra' for model alias 'nemotron-3-nano-30b-a3b'...
[10:00:06] [INFO]   |-- ✅ Passed!
[10:00:06] [INFO] 🎲 Preparing samplers to generate 10 records across 4 columns
[10:00:06] [INFO] 🧩 Generating column `product_price` from expression
[10:00:06] [INFO] llm-structured model configuration for generating column 'product_info'
[10:00:06] [INFO]   |-- model: 'nvidia/nemotron-3-nano-30b-a3b'
[10:00:06] [INFO]   |-- model alias: 'nemotron-3-nano-30b-a3b'
[10:00:06] [INFO]   |-- model provider: 'deepinfra'
[10:00:06] [INFO]   |-- inference parameters: generation_type=chat-completion, max_parallel_requests=10, extra_body={'reasoning': {'enabled': False}}, temperature=0.50, top_p=0.90, max_t

In [12]:
# More previews
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                  ┃ Value                                                                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ category              │ Tech Devices                                                                            │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ price_tens_of_dollars │ 158.67568494711531                                                                      │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ first_letter          │ E                                                                                       │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ is_hallucination      │ 1                                                                                       │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ product_price         │ 1586.7468494711532                                                                      │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ completeness_result   │ PartiallyComplete                                                                       │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ accuracy_result       │ PartiallyAccurate                                                                       │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ question              │ What makes the Eclipse Pro X1's AI-powered adaptive battery stand out in real-world     │
│                       │ usage compared to traditional smartphones?                                              │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ answer                │ The Eclipse Pro X1's AI-powered adaptive battery stands out in real-world usage by      │
│                       │ dynamically optimizing power consumption based on individual user behavior and usage    │
│                       │ patterns, extending battery life by up to 40% compared to traditional smartphones.      │
│                       │ Unlike fixed-power management systems in conventional devices, the Pro X1's AI learns   │
│                       │ when and how users interact with apps, prioritizing resources for active tasks while    │
│                       │ aggressively throttling background processes during idle periods. This results in       │
│                       │ consistently longer screen-on time—often 2–3 days on a single charge—even with heavy    │
│                       │ multitasking, without requiring manual user intervention or compromising performance.   │
│                       │ Traditional smartphones, by contrast, rely on static battery-saving modes that offer    │
│                       │ limited adaptability, leading to unpredictable performance and frequent recharging.     │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ product_info          │ {                                                                                       │
│                       │     'product_name': 'Eclipse Pro X1',                                                   │
│                       │     'key_features': [         

### View rows generated

In [13]:
final_dataset = preview.dataset.copy()
final_dataset.head()

,category,price_tens_of_dollars,first_letter,is_hallucination,product_price,product_info,question,answer,llm_answer_metrics,completeness_result,accuracy_result
0,Tech Devices,158.675685,E,1,1586.746849,"{'product_name': 'Eclipse Pro X1', 'key_featur...",What makes the Eclipse Pro X1's AI-powered ada...,The Eclipse Pro X1's AI-powered adaptive batte...,{'Completeness': {'reasoning': 'The response a...,PartiallyComplete,PartiallyAccurate
1,Office Supplies,139.910820,K,1,1399.098203,{'product_name': 'Kensington Ergonomic Adjusta...,What ergonomic benefits does the adjustable he...,**Ergonomic Benefits of the Kensington Ergonom...,{'Completeness': {'reasoning': 'The assistant ...,Complete,Accurate
2,Movies,82.518593,V,0,825.175926,{'product_name': 'VividCinema 4K Ultra HD Stre...,What key features does the VividCinema 4K Ultr...,The VividCinema 4K Ultra HD Streaming Box incl...,{'Completeness': {'reasoning': 'The response l...,Complete,Accurate
3,Music,26.504838,P,1,265.038384,"{'product_name': 'Portable Pro Studio Mixer', ...",What is the battery life of the Portable Pro S...,The Portable Pro Studio Mixer offers up to **1...,{'Completeness': {'reasoning': 'The user asked...,Complete,Accurate
4,Clothing,107.394398,X,0,1073.933977,"{'product_name': 'Xtreme Alpine Wool Coat', 'k...",What are the key benefits of the water‑repelle...,"The water‑repellent, breathable finish keeps y...",{'Completeness': {'reasoning': 'The response a...,PartiallyComplete,Accurate


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [14]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 11                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                         ┃        data type ┃             number unique values ┃        sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ category                            │           string │                        8 (80.0%) │            category │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ price_tens_of_dollars               │            float │                      10 (100.0%) │             uniform │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ first_letter                        │           string │                        8 (80.0%) │            category │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ is_hallucination                    │              int │                        2 (20.0%) │           bernoulli │
└─────────────────────────────────────┴──────────────────┴──────────────────────────────────┴─────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                  ┃               ┃                              ┃        prompt tokens ┃      completion tokens ┃
┃ column name      ┃     data type ┃         number unique values ┃           per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ question         │        string │                  10 (100.0%) │       116.5 +/- 11.9 │          22.5 +/- 15.1 │
├──────────────────┼───────────────┼──────────────────────────────┼──────────────────────┼────────────────────────┤
│ answer           │        string │                  10 (100.0%) │       101.0 +/- 63.7 │         90.5 +/- 141.6 │
└──────────────────┴───────────────┴──────────────────────────────┴──────────────────────┴────────────────────────┘
                                                                                                                   
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [15]:
job_results = data_designer_client.create(config_builder, num_records=20)

[10:00:30] [INFO] 🎨 Creating Data Designer dataset
[10:00:31] [INFO] ✅ Validation passed
[10:00:31] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[10:00:31] [INFO] 📂 Dataset path '/Users/asteiner/Documents/Git_Repos/data_designer_open_router/artifacts/dataset' already exists. Dataset from this session
		     will be saved to '/Users/asteiner/Documents/Git_Repos/data_designer_open_router/artifacts/dataset_01-16-2026_100031' instead.
[10:00:31] [INFO] 🩺 Running health checks for models...
[10:00:31] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'deepinfra' for model alias 'nemotron-3-nano-30b-a3b'...
[10:00:31] [INFO]   |-- ✅ Passed!
[10:00:31] [INFO] ⏳ Processing batch 1 of 1
[10:00:31] [INFO] 🎲 Preparing samplers to generate 20 records across 4 columns
[10:00:31] [INFO] 🧩 Generating column `product_price` from expression
[10:00:31] [INFO] llm-structured model configuration for generating column 'product_info'
[10:00:31] [INFO]   |-- model: 

### View the generated dataset

In [16]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,category,price_tens_of_dollars,first_letter,is_hallucination,product_price,product_info,question,answer,llm_answer_metrics,completeness_result,accuracy_result
0,Music,118.900118,C,0,1188.991185,{'description': 'The Crescendo Pro Audio Mixer...,What makes the Crescendo Pro Audio Mixer's tou...,The Crescendo Pro’s touchscreen with haptic fe...,"{'Accuracy': {'reasoning': ""The AI assistant's...",Complete,Accurate
1,Clothing,74.547289,Y,0,745.462887,{'description': 'Experience effortless eleganc...,What makes the Yara Silk Blend Trench Coat esp...,"The coat’s luxurious silk‑blend drape, water‑r...",{'Accuracy': {'reasoning': 'All statements in ...,PartiallyComplete,Accurate
2,Beauty,139.586271,C,0,1395.852705,"{'description': 'Experience radiant, even-tone...",What makes CeraGlow Pro Facial Serum stand out...,CeraGlow Pro Facial Serum stands out for brigh...,"{'Accuracy': {'reasoning': ""The AI assistant's...",PartiallyComplete,Accurate
3,Toiletries,123.325056,S,1,1233.240557,{'description': 'Experience the ultimate in co...,What makes SilkTouch Ultra-Soft Toilet Paper s...,SilkTouch Ultra-Soft Toilet Paper stands out b...,{'Accuracy': {'reasoning': 'The response accur...,Complete,Accurate
4,Books,63.429917,N,0,634.289171,{'description': 'A comprehensive guide that tr...,What unique feature of the Nebula Navigator he...,AR-enhanced night sky view,"{'Accuracy': {'reasoning': ""The response accur...",Complete,Accurate


### View the report

In [17]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 11                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                         ┃        data type ┃             number unique values ┃        sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ category                            │           string │                       16 (80.0%) │            category │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ price_tens_of_dollars               │            float │                      20 (100.0%) │             uniform │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ first_letter                        │           string │                       14 (70.0%) │            category │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ is_hallucination                    │              int │                        2 (10.0%) │           bernoulli │
└─────────────────────────────────────┴──────────────────┴──────────────────────────────────┴─────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                  ┃               ┃                              ┃        prompt tokens ┃      completion tokens ┃
┃ column name      ┃     data type ┃         number unique values ┃           per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ question         │        string │                  20 (100.0%) │       117.0 +/- 21.9 │           21.5 +/- 6.8 │
├──────────────────┼───────────────┼──────────────────────────────┼──────────────────────┼────────────────────────┤
│ answer           │        string │                  20 (100.0%) │       127.5 +/- 53.0 │         47.5 +/- 108.4 │
└──────────────────┴───────────────┴──────────────────────────────┴──────────────────────┴────────────────────────┘
                                                                                                                   
                                                          

### Output CSV to folder

In [18]:
from pathlib import Path

Folder_Name = "data-designer-tutorial-output"
File_Name = "dataset_OR.csv"

TUTORIAL_OUTPUT_PATH = Path(Folder_Name)
TUTORIAL_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

dataset.to_csv(TUTORIAL_OUTPUT_PATH / File_Name, index=False)